# Cell phenotyping by unsupervised clustering of the single cell data

In [3]:
# import packages
import scimap as sm
import anndata as ad

In [4]:
# Load the data that we saved in the last tutorial - (Prepare data for SCIMAP)
adata = ad.read_h5ad('/Users/aj/Dropbox (Partners HealthCare)/nirmal lab/resources/exemplarData/scimapExampleData/scimapExampleData.h5ad')